T-1日到T日结算价变化方向  logit回归  T+1日看涨/看跌行为

| 结算价变化 | 变量 |
| ---------- | ---- |
|   上涨   |  +1  |
|   下跌   |  -1  |

|持仓量变化|行为类型|变量|
|----------|--------|----|
|买持仓增加|看涨|1|
|买持仓减少|看跌|0|
|卖持仓增加|看跌|0|
|卖持仓减少|看涨|1|

In [1]:
import pymysql
from statsmodels.discrete.discrete_model import Logit
import warnings

warnings.filterwarnings('error')

conn = pymysql.connect(host='219.224.169.45', user='lizimeng', password='codegeass', db='investor', charset='utf8')
cursor = conn.cursor()

cursor.execute("use investor")

# 账号
accounts = []
sql = 'select distinct account from chicang'
cursor.execute(sql)
accounts = [row[0] for row in cursor.fetchall()]

# 日期 (昨天、今天、明天)
days = []
lastdays = []
nextdays = []
sql = "select day,lastday,nextday from commodity.trade_day where day between '20140101' and '20161231' order by day asc"
cursor.execute(sql)
for row in cursor.fetchall()[1:-1]:
    days.append(row[0])
    lastdays.append(row[1])
    nextdays.append(row[2])


for account in accounts:
    X = []
    Y = []
    for i in range(len(days)):
        day = days[i]
        lastday = lastdays[i]
        nextday = nextdays[i]
        #  今日持仓合约 或 明日持仓合约
        sql = "(select distinct vari_code,deliv_date from chicang where account=%s and tradedate=%s) union \
        (select distinct vari_code,deliv_date from chicang where account=%s and tradedate=%s)"
        cursor.execute(sql, (account,day,account,nextday))
        contracts = cursor.fetchall()
        for contract in contracts:
            # 结算价（要求昨天和今天都有结算价）
            sql = "select settle from commodity.contract_daily where day=%s and code=%s and deli=%s"
            if cursor.execute(sql, (day,contract[0],contract[1]))==0:
                continue
            settle = cursor.fetchone()[0]
            if cursor.execute(sql, (lastday,contract[0],contract[1]))==0:
                continue
            settle_last = cursor.fetchone()[0]
            if settle==settle_last:
                continue
            x = 1. if settle>settle_last else -1.
            x = (x, 1.)    # 增加截距
            # 持仓
            sql = "select sum(qty) from chicang where account=%s and tradedate=%s and vari_code=%s and deliv_date=%s and bs_flag=%s"
            for bs_flag in ('0','1'):
                # 今日持仓量
                cursor.execute(sql, (account,day,contract[0],contract[1],bs_flag))
                oi = cursor.fetchone()[0]
                if oi is None:
                    oi = 0
                # 明日持仓量
                cursor.execute(sql, (account,nextday,contract[0],contract[1],bs_flag))
                oi_next = cursor.fetchone()[0]
                if oi_next is None:
                    oi_next = 0
                # 行为分析
                if bs_flag=='0' and oi_next>oi:
                    y = 1
                elif bs_flag=='0' and oi_next<oi:
                    y = 0
                elif bs_flag=='1' and oi_next>oi:
                    y = 0
                elif bs_flag=='1' and oi_next<oi:
                    y = 1
                else:
                    continue
                X.append(x)
                Y.append(y)
    samples = len(X)
    if samples<20:
        continue
    logit = Logit(Y, X)
    try:
        result = logit.fit(maxiter=100, disp=False)
    except:
        continue
    w,b = result.params
    p_w,p_b = result.pvalues
    p1,p2 = result.predict(((1.,1.), (-1.,1.)))
    p2 = 1 - p2
    w = float(w)
    b = float(b)
    p_w = float(p_w)
    p_b = float(p_b)
    p1 = float(p1)
    p2 = float(p2)
    sql = "insert into price_change_direction2behavior (account,w,b,p_w,p_b,p1,p2,samples) values (" + "%s,"*7 + "%s)"
    cursor.execute(sql, (account,w,b,p_w,p_b,p1,p2,samples))
    conn.commit()
    # break

cursor.close()
conn.close()